# 🎓 Capstone: Fine-Tuning a Language Model for Emotion Classification

Welcome to the final workshop notebook! In this exercise, you'll fine-tune a pretrained transformer model on the **Emotion** dataset. This dataset includes short text samples labeled with one of six emotions:

- joy, sadness, anger, fear, surprise, love

You'll use all the skills you've practiced: tokenization, model loading, training, and evaluation — plus new ideas like zero-shot classification and planning your own projects.

## 📂 Part 1: Load and Explore the Dataset

In [ ]:
from datasets import load_dataset
dataset = load_dataset('emotion')
dataset['train'][0]

### 🔍 Challenge: How many examples are there per emotion?
Try using `dataset['train'].features` and simple loops or `pandas` to summarize the labels.

## ✂️ Part 2: Tokenize the Dataset

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

def tokenize(example):
    return tokenizer(example['text'], truncation=True, padding='max_length')

tokenized = dataset.map(tokenize, batched=True)
tokenized = tokenized.remove_columns(['text'])
tokenized.set_format('torch')

## 🧠 Part 3: Load Pretrained Model

In [ ]:
from transformers import AutoModelForSequenceClassification

# TODO: Load model with 6 output labels (for the 6 emotions in the dataset)
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=6
)

## 🚀 Part 4: Fine-Tune the Model

Fill in the arguments below to build your Trainer to fine-tune the model. Some parts are filled in for you already.

**NOTE:** This is a larger dataset! So make sure not to try and train for too many epochs. Start with a small number (2 or 3) and go up from there if you need to.

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

args = TrainingArguments(
    output_dir='./results',
    eval_strategy='steps', # Evaluate every eval_steps
    eval_steps=100,
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs'
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# TODO: Fill in the Trainer constructor
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

## 📈 Part 5: Evaluate and Predict

In [ ]:
def predict_emotion(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    outputs = model(**inputs)
    probs = outputs.logits.softmax(dim=-1).squeeze().tolist()
    labels = dataset['train'].features['label'].names
    predicted = labels[probs.index(max(probs))]
    print(f"\n📝 Input: {text}")
    print(f"🤖 Predicted Emotion: {predicted} ({max(probs)*100:.2f}% confidence)")
    print(f"📊 Probabilities: {dict(zip(labels, [f'{p:.3f}' for p in probs]))}")

In [ ]:
# write in your own statement to evaluate the model!
predict_emotion("I can't believe you did that!")

## 🔮 Part 6: Zero-Shot Classification Comparison

Zero-shot classification uses a model trained on **Natural Language Inference** to label text without any additional training. We'll use `facebook/bart-large-mnli`, which can reason about label descriptions.

In [ ]:
from transformers import pipeline
zero_shot = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

candidate_labels = dataset['train'].features['label'].names
text = "I can’t believe you did that!"
result = zero_shot(text, candidate_labels=candidate_labels)
print(f"\n📝 Input: {text}")
print(f"🤖 Predicted: {result['labels'][0]} ({result['scores'][0]*100:.2f}% confidence)")

### 💬 Discussion: Compare the results from your fine-tuned model and the zero-shot model. Do they agree? Does one make more sense to you than the other?

## Part 7: What’s Next?

You’ve fine-tuned a real language model and explored zero-shot reasoning. What’s next?
Here are some cool project ideas you can try:

| Task | Dataset | Link | Description |
|------|---------|------|-------------|
| Emotion Detection | `emotion` | [🔗](https://huggingface.co/datasets/emotion) | Classify tweets by emotional tone |
| Sarcasm Detection | `sarcasm` | [🔗](https://huggingface.co/datasets/sarcasm) | Detect whether something is sarcastic |
| Toxic Comments | `jigsaw_toxicity_pred` | [🔗](https://huggingface.co/datasets/jigsaw_toxicity_pred) | Label toxic or abusive language |
| Tweet Tasks | `tweet_eval` | [🔗](https://huggingface.co/datasets/tweet_eval) | Sentiment, hate, stance, emojis |
| News Topics | `ag_news` | [🔗](https://huggingface.co/datasets/ag_news) | Classify short news into topics |
| Fake News | `liar` | [🔗](https://huggingface.co/datasets/liar) | Label political statements as true/false |
| Dialogue Emotion | `daily_dialog` | [🔗](https://huggingface.co/datasets/daily_dialog) | Intent and emotion in conversations |
| Hate Speech | `hate_speech18` | [🔗](https://huggingface.co/datasets/hate_speech18) | Detect hate in text posts |

## 🌍 How to Keep Exploring

Now that you've fine-tuned and evaluated your first model, here’s how you can take your skills even further:

- 🔎 **Explore new datasets**: [Hugging Face Datasets Hub](https://huggingface.co/datasets)
- 🧠 **Try different models**: [Hugging Face Models Hub](https://huggingface.co/models)
- 📚 **Read the docs**: [Transformers Documentation](https://huggingface.co/docs/transformers)
- 💻 **Use GPUs for free**: Google Colab provides a free GPU runtime — perfect for experimenting.
- 🚀 **Share your work**: Upload your model to [Hugging Face Hub](https://huggingface.co/docs/hub) or build a simple demo with [Gradio](https://www.gradio.app/) or [Hugging Face Spaces](https://huggingface.co/spaces).

## 💡 Project Ideas by Theme

If you're looking for inspiration, here are a few directions you can take your next model:

| Theme | Ideas |
|-------|-------|
| 💬 Emotion / Mental Health | Detect mood in journaling apps, supportive chatbot for students |
| ⚠️ Content Moderation | Toxic comment filter, hate speech detector, safe content assistant |
| ✨ Creativity | Poetry classifier, sarcasm generator, meme captioner |
| 🎮 Games & Fun | NPC emotion predictor, story tone classifier, in-game chat sentiment |
| 📢 Advocacy | Fake news detector, flag biased language, classify protest messaging |
| 🗣️ Cultural NLP | Analyze slang, code-switching detection, dialect identification |